# MELODI Presto API Example Usage

In [31]:
import json
import pandas as pd
import requests
import time
from random import randint
import scipy.stats as stats

## Configure parameters

In [2]:
API_URL = "https://melodi-presto.mrcieu.ac.uk/api/"

requests.get(f"{API_URL}/status").json()

True

### How the enrichment is performed

This is a basic Fisher's exact test, using the scipy stats function.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fisher_exact.html

In [36]:
import scipy.stats as stats

queryTripleCount,queryTripleTotal,globalTripleCount,globalTripleTotal=[10,3505,147,6533824]
oddsratio, pvalue = stats.fisher_exact([[queryTripleCount, queryTripleTotal], [globalTripleCount, globalTripleTotal]])
oddsratio,pvalue

(126.81250303259678, 3.4724305806153405e-18)

### Performance example

We can compare the performance before and after the initial query. For example, if we use a query that has not already been run, e.g. `physical activity or 123`. In a PubMed search this returns over 550,000 articles. First time it takes over 20 seconds to run, creating a data set of 9,000 triples, second time should be a few seconds.

In [30]:
r=randint(0, 1000000)
q='physical activity or '+str(r)
print(q)

def run_enrich(query_term):
    start = time.time()

    endpoint = "/enrich/"
    url = f"{API_URL}{endpoint}"
    params = {
        "query": query_term,
    }
    response = requests.post(url, data=json.dumps(params))
    res = response.json()
    enrich_df = (
            pd.json_normalize(res)
    )
    print(enrich_df.shape)
    end = time.time()
    t = "{:.4f}".format(end-start)
    return t
    
t1 = run_enrich(q)
t2 = run_enrich(q)
print('t1:',t1,'\nt2:',t2)

physical activity or 113648
(9035, 16)
(9035, 16)
t1: 22.8531 
t2: 3.4203
